## load api

In [13]:
# # !pip install dotenv
# !pip install -U langchain-community
# !pip install pypdf
# !pip install faiss-cpu
# !pip install bitsandbytes
# !pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959643 sha256=f345a7777abdb6a967b249925435fd5e2b8ef73cf5c180806f4051bc94b7eba0
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
key = os.getenv('API_KEY')
# print(key[0:2])

## Load files

In [2]:
# from langchain.document_loaders import WebBaseLoader
# loader = WebBaseLoader('https://github.com/langchain-ai/langchain/blob/master/docs/docs/introduction.mdx')
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('llm_router.pdf')
document = loader.load()
print(len(document))

16


## Chunk

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
chunk = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
docs = chunk.split_documents(document)
print(docs[0].page_content)

Published as a conference paper at ICLR 2025
ROUTE LLM: L EARNING TO ROUTE LLM S WITH
PREFERENCE DATA
Isaac Ong∗1 Amjad Almahairi∗2 Vincent Wu1 Wei-Lin Chiang1 Tianhao Wu1
Joseph E. Gonzalez1 M Waleed Kadous3 Ion Stoica1,2
1UC Berkeley 2Anyscale 3Canva
ABSTRACT
Large language models (LLMs) excel at a wide range of tasks, but choosing the
right model often involves balancing performance and cost. Powerful models offer
better results but are expensive, while smaller models are more cost-effective but
less capable. To address this trade-off, we introduce a training framework for
learning efficient router models that dynamically select between a stronger and


## embedding and faiss

In [4]:

from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings()


<ipython-input-4-1f08ebd8a78e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
<ipython-input-4-1f08ebd8a78e>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingfac

In [5]:

from langchain.vectorstores import FAISS
vecdb = FAISS.from_documents(docs, embedding)
print(f"Total vectors stored: {vecdb.index.ntotal}")

Total vectors stored: 82


## retriever

In [16]:
retriever = vecdb.as_retriever(kwargs={'k':3})
query = 'which algorthims are used in llm router?'
result = retriever.get_relevant_documents(query)
print(result[0].page_content)

<ipython-input-16-c8c2bb8b6245>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = retriever.get_relevant_documents(query)


Both the matrix factorization router and causal LLM routers perform very competitively when trained
on Darena +Djudge, outperforming the commercial routing systems by achieving the same performance
with up to 40% fewer calls routed to GPT-4.
F A DDITIONAL PLOTS
We include additional plots for the results presented in Section 5.1.
15


In [25]:
print(result[1].page_content)

Published as a conference paper at ICLR 2025
million tokens. For Mixtral 8x7B, we assume the same price for both input and output tokens, which
makes the average cost $0.24 USD per 1 million tokens.
E I NDEPENDENT BENCHMARKS
Figure 2: Performance of our routers as compared to other routing systems on MT Bench. Our routers
demonstrate competitive performance, achieving stronger performance than existing routers for the
same cost.
In Figure 2, we present the performance of our best-performing routers on MT Bench as compared to
Unify AI (UnifyAI, 2024) and Martian (Martian, 2024), two existing commercial offerings for LLM
routing.


## LLM

In [ ]:
## option A

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF"
quantized_model = "tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"

# Download model (~500MB)
!wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/{model_file}

# Download model (takes ~2 mins first time)
# !wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/{quantized_model}

llm = pipeline(
    "text-generation",
    model=model_id,
    model_file=quantized_model,
    device="cpu",  # Force CPU usage
    max_new_tokens=256,
    temperature=0.3
)

--2025-04-26 18:14:39--  https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/%7Bmodel_file%7D
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.124, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-04-26 18:14:39 ERROR 404: Not Found.



config.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

ValueError: The checkpoint you are trying to load has model type `tinyllama` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [ ]:
## Option B

In [14]:

from llama_cpp import Llama

# Download model (3.8GB - smaller Q4 quantization)
model_url = "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf"
!wget {model_url} -O llama-2-7b-chat.gguf

# Initialize LLM
llm = Llama(
    model_path="llama-2-7b-chat.gguf",
    n_ctx=2048,  # Context window
    n_threads=4,  # CPU threads
    n_gpu_layers=0  # Force CPU-only
)

--2025-04-26 18:24:39--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.88, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1745695479&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTY5NTQ3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1ZjA1NWYzYTAzODg3Mzg

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
llam

## prompt

In [17]:
context  = "\n".join([d.page_content for d in result])

prompt = f"""<s>[INST] <<SYS>>
You are a RAG expert. your task is to provide a closest answer to the question below based on below context. Do not hallucinate, if you are unsure, reply 'i do not have enough context'

context: {context}
<</SYS>>

question: {query}
[/INST]
"""

print(prompt)


<s>[INST] <<SYS>>
You are a RAG expert. your task is to provide a closest answer to the question below based on below context. Do not hallucinate, if you are unsure, reply 'i do not have enough context'

context: Both the matrix factorization router and causal LLM routers perform very competitively when trained
on Darena +Djudge, outperforming the commercial routing systems by achieving the same performance
with up to 40% fewer calls routed to GPT-4.
F A DDITIONAL PLOTS
We include additional plots for the results presented in Section 5.1.
15
Published as a conference paper at ICLR 2025
million tokens. For Mixtral 8x7B, we assume the same price for both input and output tokens, which
makes the average cost $0.24 USD per 1 million tokens.
E I NDEPENDENT BENCHMARKS
Figure 2: Performance of our routers as compared to other routing systems on MT Bench. Our routers
demonstrate competitive performance, achieving stronger performance than existing routers for the
same cost.
In Figure 2, we pre

In [20]:
# Run inference
output = llm(prompt, max_tokens=150, temperature=0.3)
print(output["choices"][0]["text"])

/usr/local/lib/python3.11/dist-packages/llama_cpp/llama.py:1240: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
llama_perf_context_print:        load time =  330008.82 ms
llama_perf_context_print: prompt eval time =  330003.62 ms /   727 tokens (  453.93 ms per token,     2.20 tokens per second)
llama_perf_context_print:        eval time =   98654.62 ms /   149 runs   (  662.11 ms per token,     1.51 tokens per second)
llama_perf_context_print:       total time =  428773.10 ms /   876 tokens


Based on the provided context, it seems that the authors of the paper are comparing the performance of different LLM routers, including their own routers, against existing commercial offerings. The authors mention that their routers demonstrate competitive performance, achieving stronger performance than existing routers for the same cost. They also define evaluation metrics to capture the trade-off between cost and performance in the LLM routing problem.
Therefore, based on the context provided, I can infer that the authors are using machine learning algorithms, specifically LLMs, to build their routers. The exact algorithms used are not specified in the context, but it can be inferred that they are using some variant of a neural network or deep learning model to


## chain

In [23]:
# 1. First load your model with LlamaCpp (not raw llama_cpp.Llama)
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA

llm = LlamaCpp(
    model_path="llama-2-7b-chat.gguf",
    temperature=0.3,
    max_tokens=200,
    n_ctx=2048,
    n_threads=4,
    n_gpu_layers=0  # CPU-only
)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
llam

In [24]:

# 2. Now create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # This is now LangChain-compatible
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)
query = "which algorthims are used in llm router?"
result = qa_chain({"query": query})

<ipython-input-24-f1c21f4c7107>:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})
llama_perf_context_print:        load time =  343570.40 ms
llama_perf_context_print: prompt eval time =  343570.18 ms /   710 tokens (  483.90 ms per token,     2.07 tokens per second)
llama_perf_context_print:        eval time =   36824.83 ms /    56 runs   (  657.59 ms per token,     1.52 tokens per second)
llama_perf_context_print:       total time =  380477.76 ms /   766 tokens


In [26]:
result['result']

" I don't know the answer to your question as it is not provided in the given text. The text only mentions that the authors used matrix factorization and causal LLM routers for their experiments, but it does not specify which algorithms were used in these routers."

## using prompt template

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 1. Define your custom prompt template
template = """Use the following context to answer the question at the end.
Context contains technical details about LLM routing algorithms.
If you don't know the answer, just say you don't know.

Context: {context}

Question: {question}
Answer in technical terms with algorithm names:"""

QA_PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# 2. Create QA chain with custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_PROMPT}  # Inject custom prompt
)

# 3. Run query
query = "Which algorithms are used in LLM router?"
result = qa_chain.invoke({"query": query})  # Newer LangChain versions use invoke()

print("Answer:", result["result"])


Llama.generate: 4 prefix-match hit, remaining 800 prompt tokens to eval
